In [ ]:
!pip3 install torch torchvision torchaudio
import torch
import torchvision
# import torchaudio
print(torch.__version__)
print(torchvision.__version__)
# print(torchaudio.__version__)
from torchvision import transforms
import os
from pytorch_i3d import InceptionI3d
import videotransforms

!pip3 install av
import av
print(av.__version__)

# !pip3 install opencv-python
!pip3 install opencv-contrib-python

In [ ]:
!nvidia-smi
print(torch.cuda.is_available())

In [ ]:
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
print(device)

In [ ]:
import cv2
import os
import numpy as np

root_dir = 'short videos'
frames_root_dir = 'rgb_frames'
os.makedirs(frames_root_dir, exist_ok=True)
subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

for subdir in subdirs:
    frames_subdir = os.path.join(frames_root_dir, subdir)
    os.makedirs(frames_subdir, exist_ok=True)
    
    subsubdirs = [d for d in os.listdir(os.path.join(root_dir, subdir)) if os.path.isdir(os.path.join(root_dir, subdir, d))]
    
    for subsubdir in subsubdirs:
        frames_subsubdir = os.path.join(frames_subdir, subsubdir)
        os.makedirs(frames_subsubdir, exist_ok=True)
        
        files = os.listdir(os.path.join(root_dir, subdir, subsubdir))
        for file in files:
            if file.endswith('.mp4'):
                vidcap = cv2.VideoCapture(os.path.join(root_dir, subdir, subsubdir, file))
                success, image = vidcap.read()
                count = 1
                while success:
                    # Resize the image to 224x224
                    resize = cv2.resize(image, (224, 224))
                    # Save the frame as a JPEG file
                    cv2.imwrite(os.path.join(frames_subsubdir, subsubdir + "-%s.jpg" % str(count).zfill(6)), resize)
                    success, image = vidcap.read()
                    count += 1


In [ ]:
from denseflow import dense_flow
import os

root_dir = 'short videos'
flow_root_dir = 'flow_frames'
os.makedirs(flow_root_dir, exist_ok=True)
subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

for subdir in subdirs:
    flow_subdir = os.path.join(flow_root_dir, subdir)
    os.makedirs(flow_subdir, exist_ok=True)
    
    subsubdirs = [d for d in os.listdir(os.path.join(root_dir, subdir)) if os.path.isdir(os.path.join(root_dir, subdir, d))]
    
    for subsubdir in subsubdirs:
        flow_subsubdir = os.path.join(flow_subdir, subsubdir)
        os.makedirs(flow_subsubdir, exist_ok=True)
        
        files = os.listdir(os.path.join(root_dir, subdir, subsubdir))
        for file in files:
            if file.endswith('.mp4'):
                video_path = os.path.join(root_dir, subdir, subsubdir, file)
                video_name = subsubdir
                save_dir = flow_subsubdir
                step = 1
                bound = 15
                dense_flow_args = [video_path, video_name, save_dir, step, bound]
                dense_flow(dense_flow_args)

# TESTING

In [ ]:
base_rgb_i3d = InceptionI3d(400, in_channels=3)
base_rgb_i3d.load_state_dict(torch.load("./models/rgb_imagenet.pt"))
base_rgb_i3d.replace_logits(8)
base_rgb_i3d.eval()

In [ ]:
base_flow_i3d = InceptionI3d(400, in_channels=2)
base_flow_i3d.load_state_dict(torch.load("./models/flow_imagenet.pt"))
base_flow_i3d.replace_logits(8)
base_flow_i3d.eval()

In [ ]:
new_rgb_i3d = InceptionI3d(400, in_channels=3)
new_rgb_i3d.replace_logits(8)
new_rgb_i3d.load_state_dict(torch.load("./newmodels/rbgmodels/rgb_anamoly005000.pt", map_location=torch.device("cpu")))
new_rgb_i3d.eval()

In [ ]:
new_flow_i3d = InceptionI3d(400, in_channels=2)
new_flow_i3d.replace_logits(8)
new_flow_i3d.load_state_dict(torch.load("./newmodels/flow_anamoly005000.pt", map_location=torch.device("cpu")))
new_flow_i3d.eval()

In [43]:
rgb_root = "rgb_frames"
flow_root = "flow_frames"
vid = "Arson009_x264"
start = 1
num_frames = len(os.listdir(os.path.join(rgb_root, vid)))
print(rgb_root, vid, start, num_frames)
print(flow_root, vid, start, num_frames)

rgb_frames Arson009_x264 1 96
flow_frames Arson009_x264 1 96


In [44]:
from charades_dataset import load_rgb_frames, load_flow_frames, video_to_tensor
print(os.path.join(rgb_root, vid))
rgb_imgs = load_rgb_frames(rgb_root, vid, start, num_frames)
cropTransform = transforms.Compose([videotransforms.CenterCrop(224)])
rgb_imgs = cropTransform(rgb_imgs)
rgb_input = video_to_tensor(rgb_imgs)
print(rgb_input.shape)
rgb_input = rgb_input[None] # batch size of 1
print(rgb_input.shape)

rgb_frames\Arson009_x264
torch.Size([3, 96, 224, 224])
torch.Size([1, 3, 96, 224, 224])


In [45]:
# shape=(_BATCH_SIZE, _CHANNELS, _SAMPLE_VIDEO_FRAMES, _IMAGE_SIZE, _IMAGE_SIZE))
with torch.no_grad():
    rgb_base_prediction = base_rgb_i3d(rgb_input)
with torch.no_grad():
    rgb_prediction = new_rgb_i3d(rgb_input)

In [48]:
from charades_dataset import load_rgb_frames, load_flow_frames, video_to_tensor
print(os.path.join(flow_root, vid))
flow_imgs = load_flow_frames(flow_root, vid, start, num_frames - 1)
cropTransform = transforms.Compose([videotransforms.CenterCrop(224)])
flow_imgs = cropTransform(flow_imgs)
flow_input = video_to_tensor(flow_imgs)
print(flow_input.shape)
flow_input = flow_input[None] # batch size of 1
print(flow_input.shape)

flow_frames\Arson009_x264
torch.Size([2, 95, 224, 224])
torch.Size([1, 2, 95, 224, 224])


In [50]:
# shape=(_BATCH_SIZE, _CHANNELS, _SAMPLE_VIDEO_FRAMES, _IMAGE_SIZE, _IMAGE_SIZE))
with torch.no_grad():
    flow_base_prediction = base_flow_i3d(flow_input)
with torch.no_grad():
    flow_prediction = new_flow_i3d(flow_input)

In [51]:
print(rgb_base_prediction.shape)
print(rgb_prediction.shape)
print(flow_base_prediction.shape)
print(flow_prediction.shape)

torch.Size([1, 8, 11])
torch.Size([1, 8, 11])
torch.Size([1, 8, 11])
torch.Size([1, 8, 11])


In [ ]:
# 0: Arson
# 1: Assault
# 2: Fighting
# 3: RoadAccidents
# 4: Shooting
# 5: Stealing
# 6: Vandalism
# 7: Normal
classes = ["Arson", "Assault", "Fighting", "RoadAccidents", "Shooting", "Stealing", "Vandalism", "Normal"]

In [60]:
base_prediction = (rgb_base_prediction + flow_base_prediction)/2.0
_prediction = (rgb_prediction + flow_prediction)/2.0

In [62]:
base_predicted_class = torch.max(base_prediction, dim=2)[0].argmax(1).item()
_predicted_class = torch.max(_prediction, dim=2)[0].argmax(1).item()

print(classes[base_predicted_class])
print(classes[_predicted_class])

Stealing
Arson
